##*Surely someone has spent the time crafting the right model for the job.*
  

#Transfer learning with tensorflow part 1: Feature extraction.

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.

In [ ]:
# Are we using a gpu
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



#1. Downloading and becoming one with the data.

In [ ]:
#Get data (10% of our food class data)
import zipfile
#Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
zip_ref=zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()


--2022-11-22 11:46:33--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.128, 142.251.167.128, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   230MB/s    in 0.7s    

2022-11-22 11:46:34 (230 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
#How many images in each folder
import os
#walk through ten percent directory and list the number of files
for dir_path,dir_names,file_names in os.walk('10_food_classes_10_percent'):
 print(f'There are {len(dir_names)} directories and {len(file_names)} images in the {dir_path}')


There are 2 directories and 0 images in the 10_food_classes_10_percent
There are 10 directories and 0 images in the 10_food_classes_10_percent/test
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/sushi
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/chicken_curry
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/steak
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/ramen
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/pizza
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/chicken_wings
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/grilled_salmon
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/fried_rice
There are 0 directories and 250 images in the 10_food_classes_10_percent/test/ice_cream
There are 0 directories and 250 images in the 10_food_classes_

In [ ]:
#Each contain 75 images for training and 250 images for test as we take only the 10% of the previou training dataset.

In [ ]:
#But intuitively you might think using less data leads to less accuracy for deep  learning but transfer learning does the trick.

##Creating data loaders(preparing the data)
 

 WE'll use the ImagaDataGenerator class to load in our images in batches.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE=(224,224)
BATCH_SIZE=32
EPOCH=5
train_dir='10_food_classes_10_percent/train'
test_dir='10_food_classes_10_percent/test'
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)
print('Training_images:')
train_data_10_percent=train_datagen.flow_from_directory(train_dir,batch_size=BATCH_SIZE,target_size=IMAGE_SHAPE,
                                                        class_mode='categorical' )
print('testing_images')
test_data= test_datagen.flow_from_directory(test_dir,batch_size=BATCH_SIZE,target_size=IMAGE_SHAPE,
                                            class_mode='categorical')


Training_images:
Found 750 images belonging to 10 classes.
testing_images
Found 2500 images belonging to 10 classes.


##Setting up callbacks

Are thing to run whilst our model trains.
Callbacks are extra functionality you can add to  your models to be performed during or after training.
Some of the most popular ones are the following:

* Tracking experiments with the tensorboard callback.
* Model checkpointing with the modelcheck point callback.
* Stopping a model from training callback.

### Why do we need tensorboard callback
  Track them and compare the models.

In [ ]:
# Let's create a Tensorboard callback (functionized because we need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name,experiment_name):
  log_dir=dir_name + '/'+ experiment_name+'/'+datetime.datetime.now().strftime('%Y%m%d-%H$M%S')
  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f'saving tensorboard log files to :{log_dir}')
  return tensorboard_callback

##Creating models using tensorflow hub

In the past we've been using Tensorflow to create our own models layer by layer from **scratch!**

Now we're going to do a similar process except that the majority of our layers are goig to come from Tensorflow Hub.

We can access pre-trained models on:https://tfhub.dev/


Browsing the Tensorflow Hub page and sorting for image classification we have found the following feature model link: https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

# FEATURE EXTRACTION MODEL

In [ ]:
# But let's compare the following 2 models
resnet_url= 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'
efficientnet_url= 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [ ]:
# Import dependencies
# We're going to build the model but not from scratch
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [ ]:
# Let's make a  create model function to create a model form a URL
def create_model(model_url,num_classes):
  '''Takes a tfhub URL and creates a keras sequential moedl with it
  return a keras sequential model with model_url as feature extractor layer and 
  DEnse output layer with the num_classes output neuron'''

  #Download the pre_trained model and save it as a keras layer
  feature_extractor_layer=hub.KerasLayer(model_url,trainable=False,name='features_extraction_layer',
                                         input_shape=IMAGE_SHAPE+(3,)) # same as (224,224,3)
  #Freeze the already learned features.
  #Create our own mode
  model=tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes,activation='softmax',name='output_layer')
  ])
  return model



## Creating and testing Resnet Tensorflow Hub feature extraction model.

In [ ]:
#Create a Resnet model
resnet_model=create_model(resnet_url,10)

In [ ]:
# Then compile our model
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 features_extraction_layer (  (None, 2048)             23564800  
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
#AS we can see the 23,564,800 params are not trainable(the feature extraction model)


In [ ]:
#Compile our model
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
resnet_history= resnet_model.fit(train_data_10_percent,epochs=EPOCH,steps_per_epoch=len(train_data_10_percent),
                 validation_data=test_data,
                 validation_steps=len(test_data),callbacks=[create_tensorboard_callback('tensorflow_hub','resnetv250')])

saving tensorboard log files to :tensorflow_hub/resnetv250/20221122-11$M50
Epoch 1/5
24/24 [==============================] - 445s 19s/step - loss: 1.8594 - accuracy: 0.3693 - val_loss: 1.1665 - val_accuracy: 0.6384
Epoch 2/5
24/24 [==============================] - 420s 18s/step - loss: 0.8588 - accuracy: 0.7453 - val_loss: 0.8524 - val_accuracy: 0.7220
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 0.5956 - accuracy: 0.8480

90% training accurcy and close to 80% validation accuracy. 
**TRAINED ONLY 10% OF THE DATA BUT GOT THIS MUCH ACCURACY UNLIKE PREVIOUS MODELS, *"TRANSFER LEARNING"* IS AMAZING**

In [ ]:
import matplotlib.pyplot as plt
def plot_loss_curves(history):
  acc=history.history['accuracy']
  val_acc=history.history['val_accuracy']
  epochs=range(len(acc))
  plt.plot(epochs,acc,'r',label='training_accuracy')
  plt.plot(epochs,val_acc,'b',label="Validation_accuracy")
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  loss=history.history['loss']
  val_loss=history.history['val_loss']
  plt.plot(epochs,loss,'b',label='Training_loss')
  plt.plot(epochs,val_loss,'r',label='validation_loss')
  plt.title('Training and validation loss')
  plt.legend()

In [ ]:
plot_loss_curves(resnet_history)

###Creating and testing EfficientNetBo tennsorflow hub feature extraction model

In [ ]:
#Let's build and compare it to the resnet model
efficientnet_model=create_model(efficientnet_url,num_classes=10)
efficientnet_model.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])
efficient_history=efficientnet_model.fit(train_data_10_percent,epochs=EPOCH,steps_per_epoch=len(train_data_10_percent),
                       validation_data=test_data,validation_steps=len(test_data),
                       callbacks=[create_tensorboard_callback('tensorflow_hub','efficientnetb0')])

In [ ]:
plot_loss_curves(efficient_history)

In [ ]:
#How many layers does our effientnet b0 feature extractor have?
efficientnet_model.layers[0]
#The first layer has 309 weights
len(efficientnet_model.layers[0].weights)

#Different types of transfer learning

* Feature extraction: Freeze all parameters of the transferred model.
* Fine tuning: Unfreeze some or all of the pretrained models(transferrred model).
 


#Comparing our models using tensorboard.

We don't need to scroll for looking the training logs to compare the model..especially if we tried number of models...We could use tensorboard to compare different logs.

**NOTE:** When you upload things to tensorboard.dev, your experiments are public!

In [ ]:
#Upload Tensorboard dev records
#!tensorboard dev upload --logdir ./tensorflow_hub/ \
# --name 'EfficientNetB0 vs. Resnet50V2' \
# --description 'comparing two different TF hub feature extraction model architecturs using 10% of the training data'
# --one_shot

Our tensorboard experiments are uploaded publically here: https://tensorboard.dev/experiment/TNwY68YRREK4jxzV20T7jA/

In [ ]:
#Check out what Tensorboard experiments you have
# !tensorboard dev list

In [ ]:
#You can also delete an experiment we can using the experiment ID
# !tensorboard dev delete --experiment_id #ID NUMBER

#EXERCISES

1. Build and fit model using the same data we have but with MobileNetV2 architecture.
2. Name 3 different image classification models on TensorFlow HUb that we haven't used.
3. Build a model to classifiy images of two different things you've taken photos of.

###1. Build and fit model using the MOblieNetV2 architecture.

In [ ]:
#We have already defined the create model function so we just need the URL and num. of num_classes
#We have also created the callback tensorboard earlier.
mobilenet_url='https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5'
mobilenet_model=create_model(mobilenet_url,10)
mobilenet_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['accuracy'])
mobilenet_history=mobilenet_model.fit(train_data_10_percent,epochs=EPOCH,
                    steps_per_epoch=len(train_data_10_percent),
                    validation_data=test_data,validation_steps=len(test_data),
                    callbacks=[create_tensorboard_callback('tensorflowhub','mobilenetv2100224')])




In [ ]:
plot_loss_curves(mobilenet_history)

In [ ]:
mobilenet_model.summary()


In [ ]:
efficientnet_model.summary()

Not so great as the efficient net probably because we have used same number of trainable parameters but the no. of feature learning non trainable parameter is clearly almostc half as the Efficient net.

###2. Name 3 different image classification models other than what we just have used?

*  imagenet/mobilenet_v1_100_128/quantops/classification
* edgetpu/vision/mobilenet-edgetpu-v2-feature-vector/m
* imagenet/mobilenet_v1_025_224/quantops/classification

These are some of the many

###3. Build a model to classify two images you have taken

I have taken 10 pictures of two objects 'wuha coda' Amharic for water bottle...and another 'glue' for a glue stick ..Let's see if it can classifiy the water bootle and the glue...I am going to try it first using the efficient net.


In [ ]:
!gdown --id 1f2kYd1omRUX5KmUUPwfeiVM2Rs-Ptg_n

In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('wehacoda_and_glue.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os
for dir_path,dir_names,file_names in os.walk('wehacoda_and_glue'):
  print(f'There are {len(dir_names)} directories and {len(file_names)} images in {dir_path}')

In [ ]:
#Then we can define the training and test datasets using ImageDataGenerators and directories
train_dir2='wehacoda_and_glue/train'
test_dir2='wehacoda_and_glue/test'
from keras.preprocessing.image import ImageDataGenerator
#create instances of the train and test datagens
train_datagen2= ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen2=ImageDataGenerator(rescale=1./255)
train_data2=train_datagen2.flow_from_directory(train_dir2,batch_size=32,
                                               target_size=(224,224),
                                               class_mode='binary')
test_data2=test_datagen2.flow_from_directory(test_dir2,batch_size=32,
                                             target_size=(224,224),
                                             class_mode='binary')

In [ ]:
import pathlib
import numpy as np
data_dir=pathlib.Path('wehacoda_and_glue/train')
class_names=np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

In [ ]:
#Now let's create a new create model function because the previous one uses softmax but we want sigmoid for ours
def create_model2(model_url):
  feature_extractor_layer=hub.KerasLayer(model_url,trainable=False,
                                         name='feature_extraction_layer',
                                         input_shape=(224,224,3))
  model=tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(1,activation='sigmoid',name='output_layer')
  ])
  return model

In [ ]:
efficient_model2=create_model2(efficientnet_url)
efficient_model2.compile(loss='binary_crossentropy',
                         optimizer=tf.keras.optimizers.Adam(),
                         metrics=['accuracy'])
efficient2_history=efficient_model2.fit(train_data2,epochs=5,steps_per_epoch=len(train_data2),
                     validation_data=test_data2, validation_steps=len(test_data2))

In [ ]:
# We need to plot the loss curves
plot_loss_curves(efficient2_history)

In [ ]:
# We can then assess the curves and then evaluate it using uploaded images
def load_and_prep_image(file_name,img_shape=224):
  img=tf.io.read_file(file_name)
  img=tf.image.decode_image(img)
  print('Initial shape:',img.shape)
  img=tf.image.resize(img,size=[img_shape,img_shape])
  img=img/255
  return img
def pred_and_plot(model,file_name,class_names=class_names):
  img=load_and_prep_image(file_name)
  pred=model.predict(tf.expand_dims(img,axis=0))
  pred_class=class_names[int(tf.round(pred))]
  plt.imshow(img)
  plt.title(f'Prediction:{pred_class}')
  plt.axis(False)

In [ ]:
!gdown --id 16vaDS_6HgO6GujYQcvlLLnwuOemznmrK

In [ ]:
!gdown --id 17ov1ulQS9m-eYajS47d846cWtMGYLXoQ

In [ ]:
!gdown --id 1IXIV0G2aSI6VDkir0ffuCWo9Qhg7jTYn

In [ ]:
!gdown --id 12JZp8h_EXN44BwEzt2aBPdHgWTDBRBoS

In [ ]:
!gdown --id 1ReNtMXmADqNmRhAgV46YIqUJ-oa8zfLn

In [ ]:
!gdown --id 1SC8hFRFeule9yjXqWzNaOgL0PMPnKhfb